In [1]:
### If Google Colab
!git config --global user.email "sjs.shamsi@gmail.com"
!git config --global user.name "SJShamsi"

from google.colab import drive
drive.mount('/content/drive')

Thesis_Fies = '/content/drive/MyDrive/Thesis_Files/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from os import chdir
# chdir('/home/shoaib/Thesis/dmdt_Analysis/')
chdir(Thesis_Fies + 'Thesis/dmdt_Analysis/')
from dmdt_functions import get_differenciation, get_2Dhistogram

In [3]:
from pandas import read_parquet, DataFrame, concat
from numpy import linspace, load

In [4]:
# filtered_lightcurves_file = "/home/shoaib/PSChallenge/filtered_lightcurves.parquet"
filtered_lightcurves_file = Thesis_Fies + "/filtered_lightcurves.parquet"

# dm_bins = linspace(-0.85, 0.8, ybins)
# dt_bins = linspace(-4, 3.5, xbins)
dm_bins = load(Thesis_Fies + 'Thesis/dmdt_Analysis/Best_Bins/dm_bins_-1_1_50.npy')
dt_bins = load(Thesis_Fies + 'Thesis/dmdt_Analysis/Best_Bins/dt_bins_0_1160_50.npy')

# These files do'nt exist yet, they are save paths for later in the notebook
# dmdt_by_name_band_50x50_equidistant_bins = '/home/shoaib/Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50_equidistant_bins.pkl'
dmdt_by_name_band_50x50_equidistant_bins = Thesis_Fies + 'Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50_equidistant_bins.pkl'

In [5]:
### By experimentation we know that these are good bins

all_lightcurves = read_parquet(filtered_lightcurves_file)
# all_lightcurves = all_lightcurves.sample(frac=0.1)

In [6]:
### Looping now
grouped = all_lightcurves.groupby(['name', 'band'])
histogram_dict_list = []
num_total_groups = len(grouped)
count = 0

for (name, band), df in grouped:
    count += 1
    print(f'{count}\t/ {num_total_groups} objects in their bands done.')

    df = df.sort_values(by='mjd')
    times, mags = df['mjd'].to_numpy(), df['mag'].to_numpy()
    dtimes, dmags = get_differenciation(magnitudes=mags, times=times, log_dt=False)
    hist, _junk, _junk = get_2Dhistogram(dmagnitudes=dmags, dmagnitudes_bins=dm_bins, dtimes=dtimes, dtimes_bins=dt_bins, normalise=False, scale_factor=False)

    if hist.sum() == 0:
        continue

    object_type = df['type'].iloc[0]
    n_good_det = len(df)

    histogram_dict = {'name': name, 'type': object_type, 'band': band, 'n_good_det': n_good_det, 'histogram': hist}
    histogram_dict_list.append(histogram_dict)

Streaming output truncated to the last 5000 lines.
59079	/ 64078 objects in their bands done.
59080	/ 64078 objects in their bands done.
59081	/ 64078 objects in their bands done.
59082	/ 64078 objects in their bands done.
59083	/ 64078 objects in their bands done.
59084	/ 64078 objects in their bands done.
59085	/ 64078 objects in their bands done.
59086	/ 64078 objects in their bands done.
59087	/ 64078 objects in their bands done.
59088	/ 64078 objects in their bands done.
59089	/ 64078 objects in their bands done.
59090	/ 64078 objects in their bands done.
59091	/ 64078 objects in their bands done.
59092	/ 64078 objects in their bands done.
59093	/ 64078 objects in their bands done.
59094	/ 64078 objects in their bands done.
59095	/ 64078 objects in their bands done.
59096	/ 64078 objects in their bands done.
59097	/ 64078 objects in their bands done.
59098	/ 64078 objects in their bands done.
59099	/ 64078 objects in their bands done.
59100	/ 64078 objects in their bands done.
591

In [7]:
hist_df = DataFrame(histogram_dict_list)

In [8]:
grouped = hist_df.groupby('name')

for name, df in grouped:
    if len(df) < 2:
        continue

    hist = df['histogram'].sum()
    n_good_det = df['n_good_det'].sum()
    object_type = df['type'].iloc[0]

    histogram_dict = {'name': name, 'type': object_type, 'band': 'combined', 'n_good_det': n_good_det, 'histogram': hist}
    histogram_dict_list.append(histogram_dict)

In [9]:
hist_df = DataFrame(histogram_dict_list)
hist_df = hist_df.sort_values(by=['type', 'name', 'band'], ignore_index=True)

In [10]:
hist_df.reset_index(drop=True).to_pickle(dmdt_by_name_band_50x50_equidistant_bins)

In [11]:
hist_df

,name,type,band,n_good_det,histogram
0,1E 22044+468,A,combined,1136,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1E 22044+468,A,g,439,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,1E 22044+468,A,r,697,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,2E 1853.7+4452,A,combined,749,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,2E 1853.7+4452,A,g,158,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...
89443,J225737.49-082435.3,Seyfert_1,r,122,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
89444,J230323.47-100235.4,Seyfert_1,combined,236,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
89445,J230323.47-100235.4,Seyfert_1,g,107,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
89446,J230323.47-100235.4,Seyfert_1,r,129,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
